Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels_ = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels_ = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels_ = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels_.shape)
  print('Validation set', valid_dataset.shape, valid_labels_.shape)
  print('Test set', test_dataset.shape, test_labels_.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset,labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels_)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels_)
test_dataset, test_labels = reformat(test_dataset, test_labels_)




print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  assert tf_train_labels.shape ==(10000,10)
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases 
  assert logits.shape == (10000,10)
  l2_loss =tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+ 0.01 * l2_loss
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    #print(predictions.shape)
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 49.337807
Training accuracy: 9.9%
Validation accuracy: 13.4%
Loss at step 100: 11.760679
Training accuracy: 73.4%
Validation accuracy: 71.9%
Loss at step 200: 4.443612
Training accuracy: 79.0%
Validation accuracy: 76.8%
Loss at step 300: 1.963304
Training accuracy: 82.2%
Validation accuracy: 79.6%
Loss at step 400: 1.120528
Training accuracy: 83.6%
Validation accuracy: 81.4%
Loss at step 500: 0.830647
Training accuracy: 84.4%
Validation accuracy: 81.9%
Loss at step 600: 0.729761
Training accuracy: 84.5%
Validation accuracy: 82.2%
Loss at step 700: 0.694325
Training accuracy: 84.6%
Validation accuracy: 82.3%
Loss at step 800: 0.681783
Training accuracy: 84.6%
Validation accuracy: 82.3%
Test accuracy: 89.1%


In [7]:
#BATCH SIZE

In [8]:
batch_size = 128
hidden_node_size = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights ={ 
      'w': tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node_size])),
      'out': tf.Variable(tf.truncated_normal([hidden_node_size, num_labels]))
  }
  biases = {
      'b':tf.Variable(tf.zeros([hidden_node_size])),
      'out':tf.Variable(tf.zeros([num_labels])) 
  }
  def one_layer_relu(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      return out_layer

  pred = one_layer_relu(tf_train_dataset,weights,biases)
  pred_valid = one_layer_relu(tf_valid_dataset,weights,biases)
  pred_test = one_layer_relu(tf_test_dataset,weights,biases)
  l2_loss_relu = tf.nn.l2_loss(weights['w']) +tf.nn.l2_loss(weights['out'])
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=pred)) #+0.01 * l2_loss_relu
   
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(pred_valid)
  test_prediction = tf.nn.softmax(pred_test)

In [9]:
num_steps = 3000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 331.582886
Minibatch accuracy: 14.1%
Validation accuracy: 26.3%
Minibatch loss at step 500: 30.878056
Minibatch accuracy: 82.8%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 9.285542
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 6.497293
Minibatch accuracy: 87.5%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 3.401369
Minibatch accuracy: 90.6%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 9.025589
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Test accuracy: 90.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [10]:
batch_size = 128
hidden_node_size = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights ={ 
      'w': tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node_size])),
      'out': tf.Variable(tf.truncated_normal([hidden_node_size, num_labels]))
  }
  biases = {
      'b':tf.Variable(tf.zeros([hidden_node_size])),
      'out':tf.Variable(tf.zeros([num_labels])) 
  }
  def one_layer_relu(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      return out_layer

  pred = one_layer_relu(tf_train_dataset,weights,biases)
  pred_valid = one_layer_relu(tf_valid_dataset,weights,biases)
  pred_test = one_layer_relu(tf_test_dataset,weights,biases)
  l2_loss_relu = tf.nn.l2_loss(weights['w']) +tf.nn.l2_loss(weights['out'])
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=pred))# +0.01 * l2_loss_relu
   
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(pred_valid)
  test_prediction = tf.nn.softmax(pred_test)

In [11]:
num_steps = 3000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    train_labels_overfitting =train_labels[:256,:]
    offset = (step * batch_size) % (train_labels_overfitting.shape[0] - batch_size)
    # Generate a minibatch.
    train_dataset_overfitting = train_dataset[:256,:]
    batch_data = train_dataset_overfitting[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 438.037140
Minibatch accuracy: 10.2%
Validation accuracy: 23.6%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 56.9%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 56.9%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 56.9%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 56.9%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 56.9%
Test accuracy: 61.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
batch_size = 128
hidden_node_size = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights ={ 
      'w': tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node_size])),
      'out': tf.Variable(tf.truncated_normal([hidden_node_size, num_labels]))
  }
  biases = {
      'b':tf.Variable(tf.zeros([hidden_node_size])),
      'out':tf.Variable(tf.zeros([num_labels])) 
  }
  def one_layer_relu(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      layer = tf.nn.dropout(layer,0.6)
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      out_layer = tf.nn.dropout(out_layer, keep_prob=0.6)
      return out_layer
  
  def one_layer_relu_valid_test(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      #layer = tf.nn.dropout(layer,0.8)
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      #out_layer = tf.nn.dropout(out_layer, keep_prob=0.8)
      return out_layer
    
    
    
    
  pred = one_layer_relu(tf_train_dataset,weights,biases)
  pred_valid = one_layer_relu_valid_test(tf_valid_dataset,weights,biases)
  pred_test = one_layer_relu_valid_test(tf_test_dataset,weights,biases)
  l2_loss_relu = tf.nn.l2_loss(weights['w']) +tf.nn.l2_loss(weights['out'])
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=pred)) #+0.01 * l2_loss_relu
   
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(pred_valid)
  test_prediction = tf.nn.softmax(pred_test)

In [13]:
num_steps = 3000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    train_labels_overfitting =train_labels[:129,:]
    offset = (step * batch_size) % (train_labels_overfitting.shape[0] - batch_size)
    # Generate a minibatch.
    train_dataset_overfitting = train_dataset[:129,:]
    batch_data = train_dataset_overfitting[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the plceholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 544.092712
Minibatch accuracy: 14.1%
Validation accuracy: 21.9%
Minibatch loss at step 500: 0.464449
Minibatch accuracy: 75.8%
Validation accuracy: 65.2%
Minibatch loss at step 1000: 0.670793
Minibatch accuracy: 68.0%
Validation accuracy: 66.3%
Minibatch loss at step 1500: 0.557203
Minibatch accuracy: 68.8%
Validation accuracy: 67.2%
Minibatch loss at step 2000: 0.561036
Minibatch accuracy: 69.5%
Validation accuracy: 67.0%
Minibatch loss at step 2500: 0.528227
Minibatch accuracy: 68.8%
Validation accuracy: 66.8%
Test accuracy: 73.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [14]:
batch_size = 128
hidden_node_size = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights ={ 
      'w': tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node_size])),
      'out': tf.Variable(tf.truncated_normal([hidden_node_size, num_labels]))
  }
  biases = {
      'b':tf.Variable(tf.zeros([hidden_node_size])),
      'out':tf.Variable(tf.zeros([num_labels])) 
  }
  def one_layer_relu(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
     # layer = tf.nn.dropout(layer,0.6)
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
     # out_layer = tf.nn.dropout(out_layer, keep_prob=0.6)
      return out_layer
  
  def one_layer_relu_valid_test(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      #layer = tf.nn.dropout(layer,0.8)
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      #out_layer = tf.nn.dropout(out_layer, keep_prob=0.8)
      return out_layer
    
    
    
    
  pred = one_layer_relu(tf_train_dataset,weights,biases)
  pred_valid = one_layer_relu_valid_test(tf_valid_dataset,weights,biases)
  pred_test = one_layer_relu_valid_test(tf_test_dataset,weights,biases)
  l2_loss_relu = tf.nn.l2_loss(weights['w']) +tf.nn.l2_loss(weights['out'])
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=pred)) +0.01 * l2_loss_relu
   
    
  #learn_decay
  global_step = tf.Variable(5000) 
  learning_rate = tf.train.exponential_decay(0.5, global_step,10000, 0.96)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(pred_valid)
  test_prediction = tf.nn.softmax(pred_test)

In [15]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #train_labels_overfitting =train_labels[:129,:]
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    #train_dataset_overfitting = train_dataset[:129,:]
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the plceholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3490.904297
Minibatch accuracy: 14.1%
Validation accuracy: 19.5%
Minibatch loss at step 500: 23.643559
Minibatch accuracy: 83.6%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 0.869479
Minibatch accuracy: 85.2%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 0.655868
Minibatch accuracy: 85.2%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.618416
Minibatch accuracy: 85.9%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.785941
Minibatch accuracy: 80.5%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.816402
Minibatch accuracy: 78.1%
Validation accuracy: 83.1%
Minibatch loss at step 3500: 0.710176
Minibatch accuracy: 83.6%
Validation accuracy: 83.7%
Minibatch loss at step 4000: 0.863372
Minibatch accuracy: 79.7%
Validation accuracy: 83.6%
Minibatch loss at step 4500: 0.853196
Minibatch accuracy: 80.5%
Validation accuracy: 84.2%
Test accuracy: 90.6%


My best straight-forward CNN:
C5x4-C19x8-C5x16-P2-C7x64-P2-C3x256-P3S-C1x1024-C1x512-F2048-F64-F10
where
C5x4 = convolution with 5x5 kernel and 4 maps output
P3S = pooling with 3x3 size of type SAME

ReLU
initial weight SD: 0.05 for conv layers, Xavier for full layers
max pooling
full layer dropout 0,6
conv layer dropout 0,1
conv layer dropout before pooling
shuffle train dataset after each epoch
momentum optimizer with learning rate 0.05
batch size 2048

after 470 epochs (early stop):

train accuracy 99.6%
validation accuracy 94.6%
test accuracy 98.2%

In [42]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  #print(input_layer.shape)
  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=4,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=8,
      kernel_size=[19, 19],
      padding="same",
      activation=tf.nn.relu)
 
  conv3 = tf.layers.conv2d(
      inputs=conv2,
      filters=16,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
  pool1 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
  conv4 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[7, 7],
      padding="same",
      activation=tf.nn.relu)

  pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    
  conv5 = tf.layers.conv2d(
      inputs=pool2,
      filters=256,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
  pool3 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[3, 3], strides=2,padding='same')
        
  conv6 = tf.layers.conv2d(
      inputs=pool3,
      filters=1024,
      kernel_size=[1, 1],
      padding="same",
      activation=tf.nn.relu)
    
  conv7 = tf.layers.conv2d(
      inputs=conv6,
      filters=512,
      kernel_size=[1, 1],
      padding="same",
      activation=tf.nn.relu)
    
  flat = tf.reshape(conv7, [-1, 4 * 4 * 512])
  #print(pool2_flat.shape)
  dense1 = tf.layers.dense(inputs=flat, units=2048, activation=tf.nn.relu)
  dropout1 = tf.layers.dropout(
      inputs=dense1, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)
  dense2 = tf.layers.dense(inputs=dropout1, units=64, activation=tf.nn.relu)
  dropout2 = tf.layers.dropout(
      inputs=dense2, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)  
  logits = tf.layers.dense(inputs=dropout2, units=10)
  #assert logits.shape == (2048,10)
  

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, 
      logits=logits
  )

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.MomentumOptimizer(
        learning_rate=0.05, 
        momentum=0.9
    )
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [43]:
not_mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="not_mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'not_mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000207B5BA0D30>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [44]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [45]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_dataset},
    y= train_labels_,
    batch_size=2048,
    num_epochs=5,
    shuffle=True)
not_mnist_classifier.train(
    input_fn=train_input_fn,
    hooks=[logging_hook])



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into not_mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.10131941  0.103247    0.09972152 ...,  0.09785616  0.09806675
   0.097263  ]
 [ 0.10149713  0.10038693  0.099519   ...,  0.10074692  0.09988561
   0.09867026]
 [ 0.10143674  0.09939559  0.09832533 ...,  0.10109018  0.1005641
   0.09988732]
 ..., 
 [ 0.09955749  0.10042663  0.09936953 ...,  0.10061675  0.09781344
   0.09997325]
 [ 0.0985486   0.10462272  0.09820274 ...,  0.0984847   0.09279827
   0.10185146]
 [ 0.10226407  0.1012742   0.10046379 ...,  0.09706122  0.0987808
   0.09793749]]
INFO:tensorflow:loss = 2.30278, step = 1
INFO:tensorflow:probabilities = [[ 0.04076388  0.18731412  0.08119347 ...,  0.14167869  0.02760447
   0.06930417]
 [ 0.06779569  0.25382406  0.10678313 ...,  0.1182745   0.03077902
   0.04170142]
 [ 0.24735992  0.098703    0.0270534  ...,  0.02145586  0.15653664
   0.23442216]
 ..., 
 [ 0.16528001  0.0

In [47]:


eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": valid_dataset},
    y=valid_labels_,
    num_epochs=1,
    shuffle=False)
eval_results = not_mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-01-22-15:17:05
INFO:tensorflow:Restoring parameters from not_mnist_convnet_model\model.ckpt-489
INFO:tensorflow:Finished evaluation at 2018-01-22-15:17:06
INFO:tensorflow:Saving dict for global step 489: accuracy = 0.8964, global_step = 489, loss = 0.339431
{'accuracy': 0.89639997, 'loss': 0.33943132, 'global_step': 489}


In [48]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_dataset},
    y=test_labels_,
    num_epochs=2,
    shuffle=False)
test_results = not_mnist_classifier.evaluate(input_fn=test_input_fn)
print(test_results)

INFO:tensorflow:Starting evaluation at 2018-01-22-15:17:10
INFO:tensorflow:Restoring parameters from not_mnist_convnet_model\model.ckpt-489
INFO:tensorflow:Finished evaluation at 2018-01-22-15:17:12
INFO:tensorflow:Saving dict for global step 489: accuracy = 0.9507, global_step = 489, loss = 0.162127
{'accuracy': 0.95069999, 'loss': 0.16212671, 'global_step': 489}
